In [1]:
import time
import json
#import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import re
from collections import Counter

### Data Loading and Variables Setup

In [4]:
df = pd.read_csv('EvaluationDatasets/5000_texts.csv', delimiter=',')
df

,newspaper_id,text_id,title,chunk_id,year,city,text
0,PD1788,la-bomba-1895-01-13-page_1-1,La bomba,1,1894-1895,"Rosario, Argentina","De antemano el triunfo estaba asegurado, porqu..."
1,PF674,Image0048-page_0-0,El amolador,2,1878-1880,Bogotá,Ultimamente ha resultado que sobre ello nos da...
2,PD1540,558a32a37d1ed64f16883d01-page_0-2,Patria festiva,0,1879,Ciudad de México,El gran viajero. Ha puesto a los de su partido...
3,PD1569,558a32cc7d1ed64f168b013a-page_0-0,El coyote,1,1880,Ciudad de México,Es preciso que los señores del gobierno invent...
4,PD1087,86844de1-2628-5a12-aa7f-b796c6e62a4d_20-page_0-2,El murcielago,1,1856-1868,"Lima, Perú",Lo creo ... Sí ... ) En esto acercose a ellos ...
...,...,...,...,...,...,...,...
4995,PD1788,la-bomba-1894-04-29-page_2-2,La bomba,0,1894-1895,"Rosario, Argentina",cabo vivímos en plena corrupcion administrativ...
4996,PD1569,558a32ce7d1ed64f168b18fe-page_0-1,El coyote,2,1880,Ciudad de México,Vallarta .- Pues me quedo en In Corte para aqu...
4997,PD1087,86844de1-2628-5a12-aa7f-b796c6e62a4d_08-page_0-3,El murcielago,1,1856-1868,"Lima, Perú","Seis muchachos parió Lola, gordos, rollizos y ..."
4998,PD1087,86844de1-2628-5a12-aa7f-b796c6e62a4d_18-page_0-3,El murcielago,0,1856-1868,"Lima, Perú","extraordinario es extraordinariamente, en un, ..."


In [5]:
def compute_price(model, in_tk, out_tk):
    prices_per_1k = {
        "gpt-4o": {"input": 0.0025, "output": 0.010},
        "gpt-4o-mini": {"input": 0.00015, "output": 0.0006},
    }

    if model not in prices_per_1k:
        raise ValueError(f"Model {model} not found in the pricing list")

    input_cost = (in_tk / 1000) * prices_per_1k[model]["input"]
    output_cost = (out_tk / 1000) * prices_per_1k[model]["output"]

    return input_cost + output_cost

In [14]:
load_dotenv('./.env')
client = AzureOpenAI(

    api_version="#####",
    azure_endpoint="#####",
    api_key="#####"
)
engine = "#####"
model = "#####"

MAX_RETRIES = 2

### Request for Text Completion

In [7]:
def request(prompt, system_prompt="", max_tokens=4096, temperature=0, try_count=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :param try_count: Retry control parameter
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    #expected_tokens = count_tk(prompt)
    #if expected_tokens > 4000:
    #    return "", {"input": 0, "output": 0}, f"length - INPUT too long ({expected_tokens} tokens)"
    try:
        response = client.chat.completions.create(
            model=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}],
        )
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "content_filter":
            rsp = ""
            content_filter = response.choices[0].content_filter_results
            prompt_filter = response.prompt_filter_results[0]['content_filter_results']
            finish_reason += f" - {get_filter_flags(content_filter, prompt_filter)}"
        else:
            rsp = response.choices[0].message.content
        return rsp, { "input": response.usage.prompt_tokens, "output": response.usage.completion_tokens }, finish_reason
    except Exception as e:
        response = ""
        try: usage = { "input": response.usage.prompt_tokens, "output": response.usage.completion_tokens }
        except: usage = { "input": 0, "output": 0 }
        if "content management policy" in f"{e}":
            finish_reason = "content_filter - content_management_policy"
        else:
            if "Max retries exceeded with url" in f"{e}":
                if try_count >= MAX_RETRIES:
                    return response, usage, f"RETRYING, but reached MAX_RETRIES ({MAX_RETRIES})"
                print(f"RETRYING ({try_count})...")
                time.sleep(60)
                return request(prompt, max_tokens, temperature, try_count+1) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

def get_filter_flags(content_filter, prompt_filter):
    return ', '.join(
        [f"content.{k}" for k, v in content_filter.items() if v != {'filtered': False, 'severity': 'safe'}] +
        [f"prompt.{k}" for k, v in prompt_filter.items() if (k == 'jailbreak' and v != {'filtered': False, 'detected': False}) or (k != 'jailbreak' and v != {'filtered': False, 'severity': 'safe'})]
    )

In [15]:
RESPONSES_FILE = "EvaluationDatasets/responsesNewDataset.json"

r = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0, "fail_input_tokens": 0, "fail_output_tokens": 0, "total_price": 0, "price_per_req": 0}
if os.path.exists(RESPONSES_FILE):
    with open(RESPONSES_FILE, "r") as f:
        r = json.load(f)
else:
    with open(RESPONSES_FILE, "w") as f:
        f.write(json.dumps(r, indent=4))

In [19]:
assert r['checkpoint'] == len(r['data']), "Checkpoint does not match with executed requests"
print(f"Done {r['checkpoint']}/{len(df)} ({100*r['checkpoint']/len(df):.2f}%)")

Done 1127/5000 (22.54%)


In [10]:
system_prompt = """\
Se va a recibir un texto en español proveniente de la prensa latinoamamericana del siglo 19.
Este texto puede o no contener alguna forma de ironía, es decir, cumple con una de las siguientes situaciones:
- Presenta una contradicción entre la realidad descrita en el contexto y lo que se dice.
- Presenta una contradicción entre la realidad histórica del siglo 19 en Latinoamérica con lo que se dice
- Presenta una contradicción entre lo que se dice y el tono de como se dice (basándose en el uso de mayúsculas y signos de puntuación).

Este texto puede contener una crítica a una situación política o social contradictoria que sucedía, pero no necesariamente es una ironía, sino una opinión política negativa. 
El texto también puede contener comparaciones contradictorias o hipérboles, pero no necesariamente es una ironía, sino una expresión con lenguaje poético. Para que sea una contradicción irónica debe existir una intención de comicidad o burla en el texto, no únicamente una intención de crítica o contradicción política o una intención de descripción figurativa o poética. 

La tarea a realizar es la identificar si existe ironía o no, presente en alguna contradicción en el texto y explicar el porqué es contradictoria y cuál es la intención del autor del texto. En caso de que no exista una ironía detectada, se debe explicar el porqué no es ironía e indicar si es un texto con un sentimiento positivo, negativo o neutro. 
La respuesta debe comenzar con una de estas 4 palabras de acuerdo con lo inferido: “IRONÍA”, “POSITIVO”, “NEGATIVO” “NEUTRO”, escrito entre comillas sencillas (''). A continuación se debe añadir entre asteriscos (*) la explicación de cuál es la contradicción en caso de que sea ironía, o por qué no es ironía. 
No debes incluir nada más allá de lo que se está solicitando. La respuesta final no debe contener más de 500 palabras en total, incluyendo la descripción.\
"""

In [ ]:
for text in df.loc[r['checkpoint']:, "text"]:
    prompt = text
    #print(f"------------------------------- {r['checkpoint']+1} -----------------------------------")
    #print(prompt)
    response, usage, finish_reason = request(prompt, system_prompt)
    #print(response if response else f"ERROR: {finish_reason}")
    if finish_reason in ['stop', 'length']:
        r["input_tokens"] += usage["input"]
        r["output_tokens"] += usage["output"]
    else:
        r["fail_input_tokens"] += usage["input"]
        r["fail_output_tokens"] += usage["output"]

    r["data"].append({
        "text": text,
        "resp": response,
        "finish_reason": finish_reason
    })

    r["checkpoint"] += 1

    if r['checkpoint'] % 10 == 0:
        price = compute_price(model, r['input_tokens']+r["fail_input_tokens"], r['output_tokens']+r["fail_output_tokens"])
        r['total_price'] = round(price, 4)
        r['price_per_req'] = round(price / r['checkpoint'], 6)
        with open(RESPONSES_FILE, "w") as f:
            f.write(json.dumps(r, indent=4))
        print(f"SAVED ({r['checkpoint']})")

with open(RESPONSES_FILE, "w") as f:
    f.write(json.dumps(r, indent=4))

In [18]:
clsf = []
for s in r['data']:
    rsp = s['resp']
    if s['finish_reason'] == "stop":
        match = re.search(r"'(.*?)'", rsp)
        if match:
            extracted_word = match.group(1)
            clsf.append(extracted_word)

class_counts = Counter(clsf)
total_classes = len(clsf)
class_percentages = {cls: (count / total_classes) * 100 for cls, count in class_counts.items()}

for cls, count in class_counts.items():
    print(f"{cls}: Count = {count}, Percentage = {class_percentages[cls]:.2f}%")
    
print(f"Total: {len(clsf)} (quitando las que fallaron por el filtro de contenido)")

IRONÍA: Count = 770, Percentage = 73.68%
NEUTRO: Count = 235, Percentage = 22.49%
NEGATIVO: Count = 40, Percentage = 3.83%
Total: 1045 (quitando las que fallaron por el filtro de contenido)
